In [1]:
cd B:/Rico/Dev/GitHub/Clustering

B:\Rico\Dev\GitHub\Clustering


In [2]:
import main, cfilter, sarray, scluster, edistance
from main import *
reload(main)
reload(cfilter)
reload(sarray)
reload(scluster)
reload(edistance)
'import success'

'import success'

In [3]:
_candidate_industries = read('data/candidate_industries_04_10_2015.txt')
_job_industries = read('data/job_industries_04_10_2015.txt')
_relevant_industries = read('data/industry_of_relevant_users.txt')
_educations = read('data/educ_fields_of_relevant_users.txt')

In [4]:
def flatten(x, style='industry'):
    return smash(clean(x, style))

In [5]:
candidates = flatten(_candidate_industries)
jobs = flatten(_job_industries)
relevant = flatten(_relevant_industries)
courses = flatten(_educations, style='course')

In [6]:
len(courses)

2186

In [7]:
# Test: suffix trie
# with search function sing edit distance

reload(edistance)
from edistance import Trie
%time puno = Trie(courses)
puno.search('LINGUISTIC', k=10)
puno.insert('hello')
puno.search('ACTOR')

Wall time: 64 ms


[('ACS', 3), ('ACTING', 3), ('ART', 3), ('COE', 3), ('CPR', 3), ('ICTC', 3)]

In [8]:
import sarray
reload(sarray)
%time sa = sarray.suffix_array(courses)

Wall time: 962 ms


In [9]:
import sarray
reload(sarray)
%time sm = sa.similarity_matrix()

Wall time: 46 ms


In [10]:
import scluster
reload(scluster)
%time cluster = scluster.dense_spanning(courses, min_common=20)

Wall time: 1.12 s


In [11]:
cluster

[['COMPUTER SCIENCE INFORMATION TECHNOLOGY',
  'COMPUTER ELECTRONIC TECHNICIAN',
  'ELECTRONIC COMMUNICATION ENGINEER',
  'BUSINESS INFORMATION SYSTEM',
  'COMPUER SECRETATIAL OFFICE MANAGEMENT',
  'ELECTRICAL COMPUTER ENGINEERING SIX YEAR CURRICULUM',
  'MARKETING ADVERTISING',
  'INTER DISCIPLINARY STUDIES',
  'INTERDISCIPLINARY STUDIES TRACKS IN MANAGEMENT COMMUNICATION',
  'INTERNATIONAL BUSINESS MARKETING',
  'SCIENCE INFORMATION TECHNOLOGY',
  'COMPUTER SECRETARIAL OFFICE MANAGEMENT',
  'BUSINESS ADMINISTRATION MAJOY IN BANKING FINANCE',
  'NETWORKING TELECOMMUNICATIONS TECHNOLOGY',
  'FINANCIAL MANAGEMENT HOTEL RESTAURANT',
  'ELECTRONICS TELECOMMUNICATIONS TECHNOLOGY',
  'BUSINESS ADMINISTRATION MARKETING MANAGEMENT',
  'OFFICE INFORMATION SYSTEM',
  'INFORMATION COMMUNICATIONS TECHNOLOGY',
  'INFORMATION TECHNOLOGY IN PROGRAMMING',
  'COMPUTER SECRETARIAL BOOKKEEPING',
  'COMMUNICATIONS TECHNOLOGY MANAGEMENT',
  'INSTRUMENTAION CONTROL ENGINEERING TECHNOLOGY',
  'ELECTRONICS C

In [12]:
reload(scluster)
# remove noise
cl = scluster.lattice_spanning(courses, min_common = 3)
noise = []

def variance(data):
    if len(data) == 1: return 0
    mean = float(sum(data)) / len(data)
    var = sum(map(lambda x: (x - mean) ** 2, data)) / (len(data) - 1)
    print 'variance:', var, data[:10]
    return var

while variance(map(len,cl) + map(len,noise)) > 20:
    cl.sort(key=len, reverse=True)
    noise.extend(cl[1:])
    cl = scluster.spanning_forest(cl[0], n_clusters=len(cl[0]) / 8)


variance: 132194.314286 [1, 1, 1, 1, 2152, 1, 1, 1, 1, 1]
variance: 11490.990252 [1867, 3, 3, 3, 3, 2, 2, 2, 2, 2]
variance: 4826.64053345 [1608, 4, 4, 3, 3, 3, 3, 2, 2, 2]
variance: 2508.80995755 [1359, 4, 4, 4, 4, 3, 3, 3, 3, 3]
variance: 1386.62315809 [1120, 9, 6, 6, 5, 5, 4, 4, 4, 4]
variance: 769.091273156 [896, 17, 9, 5, 5, 4, 4, 4, 4, 4]
variance: 419.529089272 [696, 10, 8, 7, 5, 5, 5, 5, 5, 5]
variance: 242.514687571 [548, 20, 9, 9, 5, 4, 4, 4, 3, 3]
variance: 131.591604912 [413, 24, 15, 8, 6, 3, 3, 3, 3, 3]
variance: 62.9844735438 [286, 44, 12, 7, 6, 3, 3, 3, 3, 2]
variance: 22.0879489929 [131, 98, 14, 6, 2, 2, 2, 2, 2, 2]
variance: 18.6131510377 [112, 4, 2, 1, 1, 1, 1, 1, 1, 1]


In [13]:
cl

[['COMPUTER SCIENCE INFORMATION TECHNOLOGY',
  'ELECTRONIC COMMUNICATION ENGINEER',
  'SCIENCE INFORMATION TECHNOLOGY',
  'NETWORKING TELECOMMUNICATIONS TECHNOLOGY',
  'ELECTRONICS TELECOMMUNICATIONS TECHNOLOGY',
  'INFORMATION COMMUNICATIONS TECHNOLOGY',
  'INFORMATION TECHNOLOGY IN PROGRAMMING',
  'COMMUNICATIONS TECHNOLOGY MANAGEMENT',
  'INSTRUMENTAION CONTROL ENGINEERING TECHNOLOGY',
  'ELECTRONICS COMMUNICATIONS',
  'INFORMATION TECHNOLOGY GAME DESIGN DEVELOPMENT',
  'ELECTRONICS COMMUNICATIONS ENGINEERING',
  'BUSINESS INFORMATION TECHNOLOGY',
  'INFORMATION TECHNOLOGY MANAGEMENT',
  'INFORMATION COMMUNICATION COMPUTER MANAGEMENT',
  'AVIATION INFORMATION TECHNOLOGY',
  'ELECTRINOCS COMMUNICATIONS ENGINEERING',
  'INFORMATION TECHNOLOGY COMPUTER',
  'ELECTRONICS COMMUNICATIONS ENGINEERING MANAGEMENT TECHNOLOGY',
  'MECHANICAL ENGINEERING TECHNO OGY',
  'MANUFACTURING ENGINEERING MANAGEMENT',
  'INFORMATION TECHNOLOGY MANAGEMENT SYSTEM',
  'LEGAL OFFICE MANAGEMENT',
  'CHEMICAL E

In [43]:
reload(scluster)
%time dc = scluster.double_cluster(courses, min_common=4, step=1, leaf_size=100, eps=0.5, algorithm='dense')

Wall time: 39.8 s


In [44]:
dc

[[[[[[[[[[[[[[[[[['BUSINESS HOTEL RESTAURANT MANAGEMENT',
                  'HOTEL RESTAURANT MANAGER',
                  'HOTEL RESTAURANT MANAGEMENT AD',
                  'HOTEL RESTAURANT MANAGENMENT',
                  'HOSPITALITY RESTAURANT SERVICE',
                  'SCIRNCE IN HOTEL RESTAURANT MANAGEMENT',
                  'HOTEL RESTAURANT MANAGMENT',
                  'HOSPITALITY RESTAURANT MANAGEMENT',
                  'HOTEL RESTAURANT MANGEMENT',
                  'HOTEL RESTAURANT MANAGEMNET',
                  'TOURISM HOTEL RESTAURANT TECHNOLOGY',
                  'HOTEL RESTAURANT MANAGEMENT TOURISM',
                  'HOSPITALITY RESTAURANT SERVICES',
                  'TOURISM HOTEL RESTAURANT MANAGEMENT',
                  'SCIENCEIN HOTEL RESTAURANT MANAGEMENT',
                  'HOTEL RESTAURANT MANAGEENT',
                  'ASSOCIATES IN HOTEL RESTAURANTS MANAGEMENT',
                  'HOTEL RESTAURANT SERVICE TECHNOLOGY',
                  'HOTEL RESTA

In [69]:
reload(scluster)
%time labels = scluster.label_heirarchy(dc)

--------------------+ ELECTOMECHANICAL TECHNOLOGY
--------------------+ ELECTRO MECHANICAL
--------------------+ MECHANICAL TECHNOLOGY FITTER MACHINIST
--------------------+ ELECTRO MECHANICAL TECHNOLOGY ENGINEERING
--------------------+ MEDICAL TECHNOLOGY MEDTECH
--------------------+ ELECTRO MECHANICS
--------------------+ MECHANICAL TECHNOLOGY TOOL DIE
--------------------+ MECHANICAL TECHNOLOGY
--------------------+ MEDICAL TECHNOLOGY
--------------------+ OMECHANICAL TECHNOLOGY
-------------------+ MECHANICAL TECHNOLOGY
-------------------------+ BUSINESS HOTEL RESTAURANT MANAGEMENT
-------------------------+ HOTEL RESTAURANT MANAGER
-------------------------+ HOTEL RESTAURANT MANAGEMENT AD
-------------------------+ HOTEL RESTAURANT MANAGENMENT
-------------------------+ HOSPITALITY RESTAURANT SERVICE
-------------------------+ SCIRNCE IN HOTEL RESTAURANT MANAGEMENT
-------------------------+ HOTEL RESTAURANT MANAGMENT
-------------------------+ HOSPITALITY RESTAURANT MANAGEMENT


KeyboardInterrupt: 

In [46]:
reload(scluster)
%time rdc = scluster.double_cluster(relevant, min_common=4, step=1, leaf_size=100, eps=0.5, algorithm='dense')

Wall time: 18.1 s


In [47]:
rdc

[[[[[[[[[[[[[[[[[['PROFESSIONAL ORGANIZATION'],
                 ['IS A DYNAMIC PROFESSIONAL ORGANIZATION OFFERING COMPREHENSIVE SERVICES IN PROJECT DEVELOPMENT ARCHITECTURAL ENGINEERING PLANNING DESIGN GENERAL CONSTRUCTION']],
                ['GENERAL CONSTRUCTION COMPANY',
                 'ELECTRO MECHANICAL CONSTRUCTION COMPANY'],
                ['CONSTRUCTION COMPANY']],
               [[['ENGINEERING CONSTRUCTION MANAGEMENT ARCHITECTURE',
                  'ENGINEERING PROCUREMENT CONSTRUCTION MANAGEMENT',
                  'ENGINEERING CONSTRUCTION',
                  'ELECTRICAL ENGINEERING CONSTRUCTION',
                  'ENGINEERING PROCUREMENT CONSTRUCTION',
                  'GENERAL ENGINEERING CONSTRUCTION'],
                 ['CONSTRUCTION MANAGEMENT'],
                 ['ENGINEERING PROCUREMENT']],
                ['ELECTRICAL ENGINEERING']],
               [['CIVIL ENGINEERING CONTRACTING'],
                ['CONSTRUCTION CIVIL ENGINEERING MANAGEMENT']],
           